# Dormant User Sensitive Access Spike

**Dormant account suddenly accesses many sensitive files/resources; possible insider threat or credential theft.**

This notebook is a **ready-to-use SOC playbook** with:

- Triage checklist
- Splunk SPL queries
- Microsoft Sentinel KQL queries
- Elastic KQL/EQL concepts
- Containment & response commands (PowerShell / Windows / MDE)

> Replace placeholders like `<USER>`, `<HOST>`, `<DEVICE_ID>`, `<TENANT_DOMAIN>` before running.


## Triage Checklist

**Collect:** user/account, host/device, source IP, time window, impacted resources, and any correlated alerts.

**Validate:** baseline/allowlists (VPN egress, jump hosts, admin tools) before escalating.

**Preserve:** keep a copy of key logs and process trees before remediation when possible.

Focus: **dormancy**, **sensitive classification**, **access volume**, **destination shares/repos**.


## Investigation Queries

Confirm dormancy and quantify sensitive access volume; identify paths and data owners.


In [ ]:
// KQL
let window=15m;
FileEvents
| where TimeGenerated > ago(30d)
| where User == '<USER>'
| where LastSeenDays >= 30
| where DataClassification in ('Sensitive','Confidential','PII')
| summarize Accesses=count(), UniqueFiles=dcount(FilePath) by DeviceName, bin(TimeGenerated, window)
| order by Accesses desc


In [ ]:
# Splunk SPL
index=file user=<USER> last_seen_days>=30 data_classification IN (Sensitive,Confidential,PII)
| bucket _time span=15m
| stats count as accesses dc(file_path) as unique_files values(file_path) as sample_paths by host _time
| where accesses>200 OR unique_files>100


In [ ]:
# Elastic (KQL/EQL)
event.category:file AND user.name:"<USER>" AND user.last_seen_days>=30 AND data.classification:(Sensitive OR Confidential OR PII)


## Containment & Response

Suspend the account if suspicious, preserve logs, and notify data owners/compliance if required.


In [ ]:
# PowerShell
## Disable account (AD)
Disable-ADAccount -Identity <USER>

## Revoke cloud tokens (AzureAD)
Revoke-AzureADUserAllRefreshToken -ObjectId <USER_OBJECT_ID>


In [ ]:
# Bash
# Export audit logs (platform-specific). Document chain-of-custody for potential HR/legal review.
